# RF Rotations

### For a flip angle $\alpha$ around *x* or *y* axis:
$$
M' = R_x(\alpha)M =
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos\alpha & \sin\alpha \\
0 & -\sin\alpha & \cos\alpha
\end{bmatrix}
M
$$

$$
M' = R_y(\alpha)M =
\begin{bmatrix}
\cos\alpha & 0 & -\sin\alpha \\
0 & 1 & 0 \\
\sin\alpha & 0 & \cos\alpha
\end{bmatrix}
M
$$

### Flip Angle Definition:
$$
\alpha = \gamma B_1 \tau
$$

**Note:** Rotations are left-handed (also achieved with negative $\gamma$).


In [16]:
# %matplotlib inline
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider


In [17]:
gamma = 42.58e6 # Hz/T
B1 = 1e-5 # T

def rf_rotation_x(dt):
    rot_angle = -gamma * B1 * dt
    return np.array([[1,0,0], [0, np.cos(rot_angle), np.sin(rot_angle)], [0, -np.sin(rot_angle), np.cos(rot_angle)]])

def rf_rotation_y(dt):
    rot_angle = -gamma * B1 * dt
    return np.array([[np.cos(rot_angle), 0, -np.sin(rot_angle)], [0, 1, 0], [np.sin(rot_angle), 0, np.cos(rot_angle)]])

def spin_propagation(dt, rot_type: str):
    spin = np.array([0,0,1])
    if rot_type == 'x':
        matrix =  rf_rotation_x(dt)
    elif rot_type == 'y':
        matrix =  rf_rotation_y(dt)
    else:
        raise ValueError('Invalid rotation type')
    spin = matrix @ spin
    return spin

In [ ]:
rot_type = 'y'


line = np.linspace(-1, 1, 100)  # x-axis for plotting

# Initial spin vector
initial_tau = 0
initial_spin = spin_propagation(initial_tau, rot_type)
print(initial_spin)
# Create a 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(line, np.zeros(100), np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), line, np.zeros(100), 'k--', lw=1)
ax.plot(np.zeros(100), np.zeros(100), line, 'k--', lw=1)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('RF Rotation')

# Plot Bloch sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z, color='b', alpha=0.1)

# Initial quiver (arrow)
quiver = ax.quiver(0, 0, 0, initial_spin[0], initial_spin[1], initial_spin[2], 
                   color='r', arrow_length_ratio=0.2, lw=2)

# Add a slider for tau
ax_slider = plt.axes([0.2, 0.02, 0.6, 0.03], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'Time (tau)', 0, 0.0037, valinit=initial_tau, valstep=5e-5)

# Update function for the slider
def update(val):
    tau = slider.val
    spin_vector = spin_propagation(tau, rot_type)
    
    # Update the quiver by removing the old one and adding a new one
    global quiver
    quiver.remove()  # Remove the old quiver
    quiver = ax.quiver(0, 0, 0, spin_vector[0], spin_vector[1], spin_vector[2], 
                       color='r', arrow_length_ratio=0.2, lw=2)
    ax.scatter(spin_vector[0], spin_vector[1], spin_vector[2], color='r')
    fig.canvas.draw_idle()

# Connect the slider to the update function
slider.on_changed(update)

plt.show()

# RF Rotations (Arbitrary $B_1$ Phase, Axis in Transverse Plane)

### Over Time Period $\tau$:
$$
R_\phi(\alpha) =
\begin{bmatrix}
\cos^2\phi + \sin^2\phi\cos\alpha & \cos\phi\sin\phi(1 - \cos\alpha) & -\sin\phi\sin\alpha \\
\cos\phi\sin\phi(1 - \cos\alpha) & \sin^2\phi + \cos^2\phi\cos\alpha & \cos\phi\sin\alpha \\
\sin\phi\sin\alpha & -\sin\alpha\cos\phi & \cos\alpha
\end{bmatrix}
$$

### Phase Definition:
$$
\phi = \tan^{-1}\left(\frac{B_y}{B_x}\right)
$$

### Notes:
- **Rotation axis** is in the x-y plane.
- $\phi$ is an azimuth angle in *xy*-plane, while $\alpha$ is still an angle of rotation arond the new axis.
- **Formula relationship:**
  $$
  R_\phi(\alpha) = R_z(-\phi)R_x(\alpha)R_z(\phi)
  $$


# Gradient / $\Delta B_0$ Rotations

### Over Time Period $\tau$:
$$
M' = R_z(\theta)M =
\begin{bmatrix}
\cos\theta & \sin\theta & 0 \\
-\sin\theta & \cos\theta & 0 \\
0 & 0 & 1
\end{bmatrix}
M
$$

### Angle Definition:
$$
\theta = \gamma \left(\mathbf{G} \cdot \mathbf{r} + \Delta B_0\right)\tau
$$

**Note:** Rotations are left-handed (also achieved with negative $\gamma$).
